# 沸习先生学习R之data.table

  话说沸习先生这两天进行基因组的数据处理，第一次遇到比较大的数据，遇到了一个情况是需要把两个表格融合（merge）在一起，用基因可能不好解释，那我就用常见的事物来解释：大概可以理解为每一个人都有一个名字，而每个人又有身高、体重、籍贯、兴趣爱好等；而我相当于把每个人都算了一个值，得到一个表格（姓名-值），现在，我需要把身高、体重等等信息加进去。总共50w行数据，自然不能直接检索，于是我想到了哈希表，刚开始用的R，我就在R里面用hash包来完成，可是即使用哈希表之后，我发现merge也需要50h！之后我又在python里面用一模一样的方法写了一遍，用的是python里面的字典，这一次只用了1min！我的心里开始犯嘀咕：知乎上到处流传的R很慢果然是真的，R这个low货！

过了几天一个更大的数据过来了（130w），我突然想到之前在coursera的JHU的R课程上面提起过相对于data.frame，R有一个更强大的数据类型（也是一个包）data.table，于是闲来无事上网查了data.table的用法，发现里面有现成的merge函数，而且设置了key值之后一句话足矣，最重要的是130w行的数据只要不到4s！真是啪啪啪打脸打得疼：原来我只是不会用R而已。
    好，接下来沸习先生来总结一下data.table的用法。

In [1]:
# 首先，安装和调用data.table包
install.packages("data.table")
library(data.table)


  There is a binary version available but the source version is later:
             binary   source needs_compilation
data.table 1.10.4-2 1.10.4-3              TRUE



installing the source package ‘data.table’



## 一、创建data.table数据类型

### 1. data.frame类似的方法创建；

In [4]:
# data.table和data.frame类似，都可以用data.table/frame(向量名1=向量1，向量名2=向量2，...)的方法创建
dt1<-data.table(name=c("Li Lei","Han Meimei"),score=c(59,95))
print(dt1)

         name score
1:     Li Lei    59
2: Han Meimei    95


### 2.data.frame转化为data.table

In [7]:
# data.table(数据框)可以直接创建data.table
df2<-data.frame(name=c("Xiao Hong","Xiao Fang"),score=c(75,90))
dt2<-data.table(df2)
print(dt2)

        name score
1: Xiao Hong    75
2: Xiao Fang    90


### 3.fread方法

fread真的是神器！真的超级快，跟read.delim(),read.csv()之类的不是一个数量级的！我读了一个接近200M的文件只要3s，而且它可以自己推断出文件是什么分隔的，也就是说以后不用辛苦调sep参数或者辛苦记是read.table()还是read.delim()之类的了，一个fread全搞定，而且stringAsFactor默认FALSE，这点对于读很多行非常重要，还记得很多次用read.csv()忘了设置这个参数导致读几分钟也出不来，stop停不下来最后卡住重启R的厄运。

In [12]:
# fread可以将表格文件转化为data.table
setwd("~/Documents/code/GSEA/GSEA/processing")
t1<-Sys.time()
inst<-fread("GSE92742_Broad_LINCS_inst_info.txt")
t2<-Sys.time()
cat("time for fread:",t2-t1,"\n")
t1<-Sys.time()
inst2<-read.delim("GSE92742_Broad_LINCS_inst_info.txt",stringsAsFactors=F)
t2<-Sys.time()
cat("time for read.delim:",t2-t1)

time for fread: 1.280952 
time for read.delim: 16.19026

## 二、data.table的一些操作

### 1.基本操作

In [13]:
# 和data.frame有类似的操作，比如dim(),nrow(),ncol()获得行数、列数；colnames(),rownames()获得行名、列名，通过赋值还可以修改行名、列名；
# rowMeans()和colMeans()获得行平均数和列平均数；summary()和str()可以看data.table的大致情况
str(inst)

Classes ‘data.table’ and 'data.frame':	1319138 obs. of  11 variables:
 $ inst_id       : chr  "ASG001_MCF7_24H_X1_B7_DUO52HI53LO:F13" "ASG001_MCF7_24H_X1_B7_DUO52HI53LO:G13" "ASG001_MCF7_24H_X1_B7_DUO52HI53LO:I13" "ASG001_MCF7_24H_X1_B7_DUO52HI53LO:K13" ...
 $ rna_plate     : chr  "ASG001_MCF7_24H_X1" "ASG001_MCF7_24H_X1" "ASG001_MCF7_24H_X1" "ASG001_MCF7_24H_X1" ...
 $ rna_well      : chr  "F13" "G13" "I13" "K13" ...
 $ pert_id       : chr  "DMSO" "DMSO" "DMSO" "DMSO" ...
 $ pert_iname    : chr  "DMSO" "DMSO" "DMSO" "DMSO" ...
 $ pert_type     : chr  "ctl_vehicle" "ctl_vehicle" "ctl_vehicle" "ctl_vehicle" ...
 $ pert_dose     : num  0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 ...
 $ pert_dose_unit: chr  "%" "%" "%" "%" ...
 $ pert_time     : int  24 24 24 24 24 24 24 24 24 24 ...
 $ pert_time_unit: chr  "h" "h" "h" "h" ...
 $ cell_id       : chr  "MCF7" "MCF7" "MCF7" "MCF7" ...
 - attr(*, ".internal.selfref")=<externalptr> 


### 2. subset

In [15]:
# subset行是在第一个维度上面进行操作：dt[2:5,]或者是dt[2:5]，也可以用logical indexing：dt[dt$colume满足某个condition,]也可以直接
# dt[colume满足某个condition],如果满足多个条件：&连接；%in%也是一个很有用的逻辑判断符：
dt1[1]
dt1[name=="Li Lei"]
inst[inst_id %in% c("ASG001_MCF7_24H_X1_B7_DUO52HI53LO:F13","ASG001_MCF7_24H_X1_B7_DUO52HI53LO:G13")]

name,score
Li Lei,59


name,score
Li Lei,59


inst_id,rna_plate,rna_well,pert_id,pert_iname,pert_type,pert_dose,pert_dose_unit,pert_time,pert_time_unit,cell_id
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:F13,ASG001_MCF7_24H_X1,F13,DMSO,DMSO,ctl_vehicle,0.1,%,24,h,MCF7
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:G13,ASG001_MCF7_24H_X1,G13,DMSO,DMSO,ctl_vehicle,0.1,%,24,h,MCF7


In [19]:
# subset列是在第二个维度上进行操作：dt[,col]（返回向量）,dt[,.(col)]（返回data.table）,dt[,.(col1,col2)],dt[row,col]
# dt[col满足条件，.(cols)],还可以在列上使用函数
inst[,length(unique(rna_plate))]
# 在data.table包里面是list()的别称
inst[,.(n_cell=length(unique(rna_plate)),n_plate=length(unique(rna_plate)))]

[1] 3613

n_cell,n_plate
76,3613


### 3. group: by方法

接下来开始慢慢接触data.table非常有用的一部分了

In [22]:
# dt[i,j,by=]中的by可以进行分组计算，大概相当于tapply或者是split之后进行运算，但是真的比tapply和split好用很多,因为返回data.table，
# 因此j必须用.()
inst_1<-inst[,.(unique_cell_per_plate=length(unique(rna_plate))),by=rna_plate]
head(inst_1)
# .N可以用来计数：每组的个数
inst_2<-inst[,.(.N,unique_cell_per_plate=length(unique(rna_plate))),by=rna_plate]
head(inst_2)

rna_plate,unique_cell_per_plate
ASG001_MCF7_24H_X1,1
ASG001_MCF7_6H_X2,1
ASG001_PC3_24H_X2,1
ASG001_PC3_24H_X4,1
ASG001_PC3_24H_X5,1
ASG001_PC3_6H_X3,1


rna_plate,N,unique_cell_per_plate
ASG001_MCF7_24H_X1,341,1
ASG001_MCF7_6H_X2,351,1
ASG001_PC3_24H_X2,350,1
ASG001_PC3_24H_X4,358,1
ASG001_PC3_24H_X5,358,1
ASG001_PC3_6H_X3,339,1
